In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import shutil
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
_CSV_COLUMNS = [
    '', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 
    'target', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
]

_CSV_COLUMN_DEFAULTS = [[0], ['missing_msno'], ['missing_song_id'], ['missing_tab'], ['missing_name'], ['missing_type'], 
                        [''], [241812.0], ['missing_genre_ids'], ['missing_artist_name'], ['missing_composer'], ['missing_lyricist'], 
                        [-1.0], [0], [21], ['missing_gender'], [0], [0], [0]]

parser = argparse.ArgumentParser()

parser.add_argument(
    '--model_dir', type=str, default='/tmp/census_model',
    help='Base directory for the model.')

parser.add_argument(
    '--model_type', type=str, default='deep',#'wide_deep',
    help="Valid model types: {'wide', 'deep', 'wide_deep'}.")

parser.add_argument(
    '--train_epochs', type=int, default=40, help='Number of training epochs.')

parser.add_argument(
    '--epochs_per_eval', type=int, default=2,
    help='The number of training epochs to run between evaluations.')

parser.add_argument(
    '--batch_size', type=int, default=40, help='Number of examples per batch.')

parser.add_argument(
    '--train_data', type=str, default='./labeled_train_X2.csv',
    help='Path to the training data.')

parser.add_argument(
    '--test_data', type=str, default='./labeled_train_X3.csv',
    help='Path to the test data.')

_StoreAction(option_strings=['--test_data'], dest='test_data', nargs=None, const=None, default='./labeled_train_X3.csv', type=<class 'str'>, choices=None, help='Path to the test data.', metavar=None)

In [3]:
_NUM_EXAMPLES = {
    'train': 5901935 + 1474484,
    'validation': 1000,
}


In [4]:
def build_model_columns():
  """Builds a set of wide and deep feature columns."""
  # Continuous columns
  bd = tf.feature_column.numeric_column('bd')
  song_length = tf.feature_column.numeric_column('song_length')
  city = tf.feature_column.numeric_column('city')
  language = tf.feature_column.numeric_column('language')
  registered_via = tf.feature_column.numeric_column('registered_via')
  genre_ids = tf.feature_column.categorical_column_with_hash_bucket(
      'genre_ids', hash_bucket_size=10000)

  gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'gender', ['male', 'female'])

  msno = tf.feature_column.categorical_column_with_hash_bucket(
      'msno', hash_bucket_size=3200000)
  song_id = tf.feature_column.categorical_column_with_hash_bucket(
      'song_id', hash_bucket_size=2000000)
  artist_name = tf.feature_column.categorical_column_with_hash_bucket(
      'artist_name', hash_bucket_size=100000)
  composer = tf.feature_column.categorical_column_with_hash_bucket(
      'composer', hash_bucket_size=100000)
  lyricist = tf.feature_column.categorical_column_with_hash_bucket(
      'lyricist', hash_bucket_size=100000)
  # To show an example of hashing:
  source_system_tab = tf.feature_column.categorical_column_with_hash_bucket(
      'source_system_tab', hash_bucket_size=20)
  source_screen_name = tf.feature_column.categorical_column_with_hash_bucket(
      'source_screen_name', hash_bucket_size=20)
  source_type = tf.feature_column.categorical_column_with_hash_bucket(
      'source_type', hash_bucket_size=20)

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(
      bd, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
  song_length_buckets = tf.feature_column.bucketized_column(
      song_length, boundaries=[160000, 320000, 480000, 640000])

  # Wide columns and deep columns.
  base_columns = [
      gender, artist_name, source_system_tab, source_screen_name,
      source_type, composer, lyricist, age_buckets, song_id, msno,
  ]

  crossed_columns = [
      tf.feature_column.crossed_column(
          ['artist_name', 'composer', 'lyricist'], hash_bucket_size=10000),
  ]

  wide_columns = base_columns + crossed_columns

  deep_columns = [
      bd,
      city,
      language,
      tf.feature_column.numeric_column('song_length', normalizer_fn=lambda x: (x - 241812.0) / 67351),
      registered_via,
      tf.feature_column.embedding_column(genre_ids, 9),
      tf.feature_column.embedding_column(artist_name, 12),
      tf.feature_column.embedding_column(composer, 12),
      tf.feature_column.embedding_column(lyricist, 12),
      tf.feature_column.embedding_column(source_system_tab, 9),
      tf.feature_column.embedding_column(source_screen_name, 9),
      tf.feature_column.embedding_column(source_type, 9),
  ]

  return wide_columns, deep_columns

In [5]:
def build_estimator(model_dir, model_type):
  """Build an estimator appropriate for the given model type."""
  wide_columns, deep_columns = build_model_columns()
  hidden_units = [128, 128, 64, 64, 32]#[100, 75, 50, 25]

  # Create a tf.estimator.RunConfig to ensure the model is run on CPU, which
  # trains faster than GPU for this model.
  run_config = tf.estimator.RunConfig().replace(
      session_config=tf.ConfigProto(device_count={'GPU': 0}))

  if model_type == 'wide':
    return tf.estimator.LinearClassifier(
        model_dir=model_dir,
        feature_columns=wide_columns,
        config=run_config)
  elif model_type == 'deep':
    return tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        config=run_config)
  else:
    return tf.estimator.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units,
        config=run_config)

In [6]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, field_delim = ',', record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('target')
    return features, tf.equal(labels, '1')

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)
  
  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5) #skip(7376419)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  return features, labels

In [8]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [20]:
print("what's up?")

In [9]:
temp = 0
temp2 = 0
model = None
def main(unused_argv):
  global temp
  global temp2
  global model
  # Clean up the model directory if present
  shutil.rmtree(FLAGS.model_dir, ignore_errors=True)
  model = build_estimator(FLAGS.model_dir, FLAGS.model_type)

  # Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
  for n in range(2):#range(FLAGS.train_epochs // FLAGS.epochs_per_eval):
    model.train(input_fn=lambda: input_fn(
        FLAGS.train_data, FLAGS.epochs_per_eval, True, FLAGS.batch_size))

    results = model.evaluate(input_fn=lambda: input_fn(
        FLAGS.test_data, 1, False, FLAGS.batch_size))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * FLAGS.epochs_per_eval)
    print('-' * 60)

    #for key in sorted(results):
    #  print('%s: %s' % (key, results[key]))
  #model.export_savedmodel(export_dir_base='./')
  column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
  df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                            skip_blank_lines=True, keep_default_na=False)#, dtype={'city':str, 'language':str})
  predictions = list(model.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
      x=df, num_epochs=1,shuffle=False)))
  res = pd.DataFrame(predictions)
  temp = model
  temp2 = predictions
  res.to_csv('pred.csv', sep = ',', header = False)
  

if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed + [temp])

INFO:tensorflow:Using config: {'_model_dir': '/tmp/census_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff48a5aedd8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Parsing ./labeled_train_X2.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/census_model/model.ckpt.
INFO:tensorflow:loss = 28.7683, step = 1
INFO:tensorflow:global_step/sec: 162.493
INFO:tensorflow:loss = 27.7332, step = 101 (0.617 sec)
INFO:tensorflow:global_step/sec: 246.429
INFO:tensorflow:loss = 27.0788, step = 201 (0.406 sec)
INFO:tensorflow:global_step/sec: 253.62
INFO:ten

INFO:tensorflow:global_step/sec: 253.175
INFO:tensorflow:loss = 28.5408, step = 7801 (0.394 sec)
INFO:tensorflow:global_step/sec: 258.28
INFO:tensorflow:loss = 28.8442, step = 7901 (0.387 sec)
INFO:tensorflow:global_step/sec: 259.919
INFO:tensorflow:loss = 23.8146, step = 8001 (0.385 sec)
INFO:tensorflow:global_step/sec: 258.543
INFO:tensorflow:loss = 27.8653, step = 8101 (0.387 sec)
INFO:tensorflow:global_step/sec: 256.578
INFO:tensorflow:loss = 26.6391, step = 8201 (0.390 sec)
INFO:tensorflow:global_step/sec: 259.073
INFO:tensorflow:loss = 26.8062, step = 8301 (0.386 sec)
INFO:tensorflow:global_step/sec: 258.398
INFO:tensorflow:loss = 22.8271, step = 8401 (0.387 sec)
INFO:tensorflow:global_step/sec: 255.947
INFO:tensorflow:loss = 28.1224, step = 8501 (0.391 sec)
INFO:tensorflow:global_step/sec: 259.484
INFO:tensorflow:loss = 27.6262, step = 8601 (0.385 sec)
INFO:tensorflow:global_step/sec: 258.878
INFO:tensorflow:loss = 28.5318, step = 8701 (0.386 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:loss = 24.1252, step = 16201 (0.379 sec)
INFO:tensorflow:global_step/sec: 263.75
INFO:tensorflow:loss = 25.502, step = 16301 (0.379 sec)
INFO:tensorflow:global_step/sec: 266.119
INFO:tensorflow:loss = 25.7254, step = 16401 (0.376 sec)
INFO:tensorflow:global_step/sec: 266.198
INFO:tensorflow:loss = 23.9685, step = 16501 (0.376 sec)
INFO:tensorflow:global_step/sec: 266.643
INFO:tensorflow:loss = 27.1719, step = 16601 (0.375 sec)
INFO:tensorflow:global_step/sec: 267.583
INFO:tensorflow:loss = 24.5916, step = 16701 (0.374 sec)
INFO:tensorflow:global_step/sec: 264.097
INFO:tensorflow:loss = 26.2967, step = 16801 (0.378 sec)
INFO:tensorflow:global_step/sec: 266.9
INFO:tensorflow:loss = 26.2125, step = 16901 (0.375 sec)
INFO:tensorflow:global_step/sec: 264.535
INFO:tensorflow:loss = 26.4386, step = 17001 (0.378 sec)
INFO:tensorflow:global_step/sec: 265.093
INFO:tensorflow:loss = 28.8327, step = 17101 (0.377 sec)
INFO:tensorflow:global_step/sec: 267.294
INFO:tensorflow:loss = 2

INFO:tensorflow:loss = 26.5018, step = 24601 (0.383 sec)
INFO:tensorflow:global_step/sec: 263.239
INFO:tensorflow:loss = 26.2578, step = 24701 (0.381 sec)
INFO:tensorflow:global_step/sec: 255.252
INFO:tensorflow:loss = 26.2043, step = 24801 (0.391 sec)
INFO:tensorflow:global_step/sec: 263.161
INFO:tensorflow:loss = 26.5516, step = 24901 (0.380 sec)
INFO:tensorflow:global_step/sec: 257.205
INFO:tensorflow:loss = 25.8018, step = 25001 (0.389 sec)
INFO:tensorflow:global_step/sec: 259.955
INFO:tensorflow:loss = 22.393, step = 25101 (0.385 sec)
INFO:tensorflow:global_step/sec: 261.989
INFO:tensorflow:loss = 25.76, step = 25201 (0.382 sec)
INFO:tensorflow:global_step/sec: 258.912
INFO:tensorflow:loss = 26.9482, step = 25301 (0.386 sec)
INFO:tensorflow:global_step/sec: 254.752
INFO:tensorflow:loss = 22.2514, step = 25401 (0.393 sec)
INFO:tensorflow:global_step/sec: 265.537
INFO:tensorflow:loss = 24.5299, step = 25501 (0.377 sec)
INFO:tensorflow:global_step/sec: 261.262
INFO:tensorflow:loss = 

INFO:tensorflow:loss = 26.1649, step = 33001 (0.385 sec)
INFO:tensorflow:global_step/sec: 259.306
INFO:tensorflow:loss = 24.5091, step = 33101 (0.386 sec)
INFO:tensorflow:global_step/sec: 257.665
INFO:tensorflow:loss = 25.9256, step = 33201 (0.388 sec)
INFO:tensorflow:global_step/sec: 258.194
INFO:tensorflow:loss = 23.9831, step = 33301 (0.387 sec)
INFO:tensorflow:global_step/sec: 256.784
INFO:tensorflow:loss = 25.6791, step = 33401 (0.389 sec)
INFO:tensorflow:global_step/sec: 265.067
INFO:tensorflow:loss = 21.3004, step = 33501 (0.377 sec)
INFO:tensorflow:global_step/sec: 258.09
INFO:tensorflow:loss = 26.4608, step = 33601 (0.387 sec)
INFO:tensorflow:global_step/sec: 257.23
INFO:tensorflow:loss = 23.3238, step = 33701 (0.389 sec)
INFO:tensorflow:global_step/sec: 259.639
INFO:tensorflow:loss = 27.5567, step = 33801 (0.385 sec)
INFO:tensorflow:global_step/sec: 260.256
INFO:tensorflow:loss = 23.8596, step = 33901 (0.384 sec)
INFO:tensorflow:global_step/sec: 261.663
INFO:tensorflow:loss =

...

INFO:tensorflow:loss = 24.6081, step = 721772 (0.391 sec)
INFO:tensorflow:global_step/sec: 256.233
INFO:tensorflow:loss = 23.0323, step = 721872 (0.390 sec)
INFO:tensorflow:global_step/sec: 257.822
INFO:tensorflow:loss = 20.7414, step = 721972 (0.388 sec)
INFO:tensorflow:global_step/sec: 256.326
INFO:tensorflow:loss = 21.5765, step = 722072 (0.390 sec)
INFO:tensorflow:global_step/sec: 255.776
INFO:tensorflow:loss = 26.9447, step = 722172 (0.391 sec)
INFO:tensorflow:global_step/sec: 256.782
INFO:tensorflow:loss = 23.6147, step = 722272 (0.389 sec)
INFO:tensorflow:global_step/sec: 258.283
INFO:tensorflow:loss = 21.9891, step = 722372 (0.387 sec)
INFO:tensorflow:global_step/sec: 257.074
INFO:tensorflow:loss = 26.2715, step = 722472 (0.390 sec)
INFO:tensorflow:global_step/sec: 256.321
INFO:tensorflow:loss = 23.7495, step = 722572 (0.389 sec)
INFO:tensorflow:global_step/sec: 256.693
INFO:tensorflow:loss = 25.2552, step = 722672 (0.390 sec)
INFO:tensorflow:global_step/sec: 253.987
INFO:tenso

INFO:tensorflow:loss = 28.7095, step = 730072 (0.422 sec)
INFO:tensorflow:global_step/sec: 241.342
INFO:tensorflow:loss = 25.2251, step = 730172 (0.414 sec)
INFO:tensorflow:global_step/sec: 242.145
INFO:tensorflow:loss = 21.031, step = 730272 (0.414 sec)
INFO:tensorflow:global_step/sec: 238.556
INFO:tensorflow:loss = 24.4241, step = 730372 (0.418 sec)
INFO:tensorflow:global_step/sec: 236.281
INFO:tensorflow:loss = 23.553, step = 730472 (0.423 sec)
INFO:tensorflow:global_step/sec: 235.312
INFO:tensorflow:loss = 25.0799, step = 730572 (0.425 sec)
INFO:tensorflow:global_step/sec: 239.385
INFO:tensorflow:loss = 24.7865, step = 730672 (0.418 sec)
INFO:tensorflow:global_step/sec: 236.499
INFO:tensorflow:loss = 23.5962, step = 730772 (0.423 sec)
INFO:tensorflow:global_step/sec: 242.694
INFO:tensorflow:loss = 26.6967, step = 730872 (0.412 sec)
INFO:tensorflow:global_step/sec: 240.116
INFO:tensorflow:loss = 25.4583, step = 730972 (0.417 sec)
INFO:tensorflow:global_step/sec: 236.445
INFO:tensorf

Results at epoch 4
------------------------------------------------------------
INFO:tensorflow:Restoring parameters from /tmp/census_model/model.ckpt-737742


SystemExit: 

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                        skip_blank_lines=True, keep_default_na=False)
predictions = list(temp.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
  x=df, num_epochs=1,shuffle=False)))
res = pd.DataFrame(predictions)

In [26]:

column_names3=['id', 'target', 'score', 's1', 's2', 's']
def conv3(val):  
    return np.int(val[1])
census_pred = pd.read_csv('./pred.csv', index_col=False, names=column_names3,
                          skip_blank_lines=True, keep_default_na=False, converters={'target':conv3})

In [27]:
header=['id', 'target']
census_pred.to_csv('output.csv', index=False, columns = header)

In [40]:
np.sum(census_pred['target'])

84351